# Imports

In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

# Load data

In [3]:
df = pd.read_csv('data/ready/train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,0,65.0,8450,0,0,0,0,0,...,0,0,0,0,0,2,2008,0,0,208500
1,2,20,0,80.0,9600,0,0,0,0,0,...,0,0,0,0,0,5,2007,0,0,181500
2,3,60,0,68.0,11250,0,0,1,0,0,...,0,0,0,0,0,9,2008,0,0,223500
3,4,70,0,60.0,9550,0,0,1,0,0,...,0,0,0,0,0,2,2006,0,1,140000
4,5,60,0,84.0,14260,0,0,1,0,0,...,0,0,0,0,0,12,2008,0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,0,62.0,7917,0,0,0,0,0,...,0,0,0,0,0,8,2007,0,0,175000
1456,1457,20,0,85.0,13175,0,0,0,0,0,...,0,0,1,0,0,2,2010,0,0,210000
1457,1458,70,0,66.0,9042,0,0,0,0,0,...,0,0,3,1,2500,5,2010,0,0,266500
1458,1459,20,0,68.0,9717,0,0,0,0,0,...,0,0,0,0,0,4,2010,0,0,142125


# Train-val split

In [4]:
df_train, df_val = train_test_split(df, test_size=0.2)
df_train.shape, df_val.shape

((1168, 81), (292, 81))

In [5]:
cols_x = list(df_train.drop(columns=['Id', 'SalePrice']).columns)
cols_y = 'SalePrice'
cols_x[:5]

['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street']

# Logit target

In [6]:
def one_plus_log(x):
    return np.log(1 + x)

def one_plus_log_reverse(x):
    return np.exp(x) - 1

df_train['SalePrice'] = df_train['SalePrice'].apply(one_plus_log)
df_val['SalePrice'] = df_val['SalePrice'].apply(one_plus_log)
df_val.head()

/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_93513/3151798132.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['SalePrice'] = df_train['SalePrice'].apply(one_plus_log)
/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_93513/3151798132.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['SalePrice'] = df_val['SalePrice'].apply(one_plus_log)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
383,384,45,4,60.0,9000,0,0,0,0,0,...,0,0,0,0,0,10,2009,0,0,11.238502
1334,1335,160,1,24.0,2368,0,0,0,0,0,...,0,0,0,0,0,5,2009,0,0,11.736077
981,982,60,0,98.0,12203,0,0,1,0,0,...,0,0,0,0,0,7,2009,0,0,12.724869
934,935,20,0,313.0,27650,0,0,2,3,0,...,0,0,0,0,0,11,2008,0,0,12.396697
856,857,80,0,-1.0,10970,0,0,1,2,0,...,0,0,1,0,0,10,2008,0,0,11.898195


# Apply model

In [7]:
def rmse(y, y_hat):
    return mean_squared_error(y, y_hat) ** 0.5

min samples leaf

In [19]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=1)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.0808155624751703
Test RMSE 0.11338020778163703


In [20]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=3)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.08271302609386146
Test RMSE 0.11532430733242154


In [21]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=5)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.08544690175726018
Test RMSE 0.11467528942963325


In [22]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.08848056814652173
Test RMSE 0.11340724535834193


In [23]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=20)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.09475310001653588
Test RMSE 0.11645856951952153


In [24]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=50)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.10618286687969794
Test RMSE 0.11913746074228353


max features

In [25]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=0.1)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.10252681334231276
Test RMSE 0.11655362179114714


In [26]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=0.3)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.09330138318350573
Test RMSE 0.1163532755522098


In [27]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=0.5)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.09148460605661687
Test RMSE 0.11653947658534985


In [28]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=0.7)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.09026066859565629
Test RMSE 0.11451581248017013


In [29]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=0.9)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.09096352614605889
Test RMSE 0.11460189483372248


In [31]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=1.)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.08848056814652173
Test RMSE 0.11340724535834193


learning rate

In [38]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=1., learning_rate=0.1)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.08848056814652173
Test RMSE 0.11340724535834193


In [39]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=1., learning_rate=0.01)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.22549611426358124
Test RMSE 0.22040105962266132


In [40]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=1., learning_rate=0.3)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.06101384646434209
Test RMSE 0.12720378289701376


In [41]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=1., learning_rate=0.2)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.06974664352620964
Test RMSE 0.11600111747327001


In [42]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=1., learning_rate=0.05)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.10792812473477738
Test RMSE 0.1205587307906662


cpp alpha

In [44]:
gb = GradientBoostingRegressor(random_state=27
                               , min_samples_leaf=10, max_features=1.
                               , learning_rate=0.1, ccp_alpha=0.5)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.4020433455353874
Test RMSE 0.38841976873972806


In [45]:
gb = GradientBoostingRegressor(random_state=27
                               , min_samples_leaf=10, max_features=1.
                               , learning_rate=0.1, ccp_alpha=0.01)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.23900526808807324
Test RMSE 0.22543767553151717


In [49]:
gb = GradientBoostingRegressor(random_state=27
                               , min_samples_leaf=10, max_features=1.
                               , learning_rate=0.1, ccp_alpha=0.0001)

gb.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = gb.predict(df_train[cols_x])
y_val_hat = gb.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.10243139436124449
Test RMSE 0.11751672816355171


# Predict test set

In [50]:
df_test = pd.read_csv('data/ready/test.csv')
df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,4,80.0,11622,0,-1,0,0,0,...,120,0,-1,1,-1,0,6,2010,0,0
1,1462,20,0,81.0,14267,0,-1,1,0,0,...,0,0,-1,-1,2,12500,6,2010,0,0
2,1463,60,0,74.0,13830,0,-1,1,0,0,...,0,0,-1,1,-1,0,3,2010,0,0
3,1464,60,0,78.0,9978,0,-1,1,0,0,...,0,0,-1,-1,-1,0,6,2010,0,0
4,1465,120,0,43.0,5005,0,-1,1,3,0,...,144,0,-1,-1,-1,0,1,2010,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,1,21.0,1936,0,-1,0,0,0,...,0,0,-1,-1,-1,0,6,2006,0,0
1455,2916,160,1,21.0,1894,0,-1,0,0,0,...,0,0,-1,-1,-1,0,4,2006,0,1
1456,2917,20,0,160.0,20000,0,-1,0,0,0,...,0,0,-1,-1,-1,0,9,2006,0,1
1457,2918,85,0,62.0,10441,0,-1,0,0,0,...,0,0,-1,1,1,700,7,2006,0,0


In [52]:
df['SalePrice'] = df['SalePrice'].apply(one_plus_log)

In [53]:
gb = GradientBoostingRegressor(random_state=27, min_samples_leaf=10, max_features=1., learning_rate=0.1)

gb.fit(df[cols_x], df[cols_y])

df_test['SalePrice'] = gb.predict(df_test[cols_x])
df_submission = df_test[['Id', 'SalePrice']]

df_submission

,Id,SalePrice
0,1461,11.688879
1,1462,11.938192
2,1463,12.088112
3,1464,12.149397
4,1465,12.161981
...,...,...
1454,2915,11.312756
1455,2916,11.313046
1456,2917,12.100013
1457,2918,11.719498


In [54]:
df_submission['SalePrice'] = df_submission['SalePrice'].apply(one_plus_log_reverse)
df_submission

/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_93513/3176596026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submission['SalePrice'] = df_submission['SalePrice'].apply(one_plus_log_reverse)


,Id,SalePrice
0,1461,119237.271681
1,1462,152998.820872
2,1463,177745.167917
3,1464,188979.177761
4,1465,191372.284130
...,...,...
1454,2915,81858.229192
1455,2916,81881.908111
1456,2917,179873.204925
1457,2918,122944.667499


In [55]:
df_submission.to_csv('data/results/gb_sklearn.csv', index=False)